In [ ]:
import json
import os
import utils
import numpy as np

## Extract the ground truth and inference bounding boxes from the Json.
if(os.path.isdir('jsons')):
    gtruth_num = -1
    out_num = -1
    for element in os.listdir('jsons'):
        for file in os.listdir('jsons/'+element):
            f = open('jsons/'+element+'/'+file)
            data = json.load(f)
            gtruth = list()
            glabels = list()
            out = list()
            outlabels = list()

            temp = np.asarray(data['output']['label'])

            for i in range(0, temp.size):
                gtruth += [temp[i]['bbox_3d']]
                glabels += [temp[i]['label_3d']]

            temp = np.asarray(data['output']['inference'])

            for i in range(0, temp.size):
                out += [temp[i]['bbox_3d']]
                outlabels += [temp[i]['label_3d']]

            for bb in gtruth:
                bb[3], bb[4] = bb[4], bb[3]

            file = file.split(".")[0] + '0'
            points = utils.load_point_clouds('/theia/Theia-SP4/olivia_202208/'+element+'/pcd_files/LidarVeloRoofFC/split_000000/'+file+'.pcd')
            label = 1

            ## Use the bounding boxes to extract the ground truth.

            gtruth = np.asarray(gtruth)
            glabels = np.asarray(glabels)

            gtruth_boxes = gtruth[np.where(glabels==label)]

            if len(gtruth_boxes != 0):
                corners3d = utils.boxes_to_corners_3d(gtruth_boxes)
                points_flag = utils.is_within_3d_box(points, corners3d)

                points_is_within_3d_box = []
                for i in range(len(points_flag)):
                    p = points[points_flag[i]]
                    if len(p)>0:
                        points_is_within_3d_box.append(p)
                        box = gtruth_boxes[i]
                        points_canonical, box_canonical = utils.points_to_canonical(p, box)
                        gtruth_num += 1
                        pts_name = 'extract/gtruth_pts/'+str(label)+'/'+str(gtruth_num)+'.'+element+'.'+file
                        box_name = 'extract/gtruth_boxes/'+str(label)+'/'+str(gtruth_num)+'.'+element+'.'+file
                        corner_name = 'extract/gtruth_corners/'+str(label)+'/'+str(gtruth_num)+'.'+element+'.'+file

                        if (os.path.exists('extract/gtruth_pts/'+str(label)) == False):
                            os.makedirs('extract/gtruth_pts/'+str(label))
                        if (os.path.exists('extract/gtruth_boxes/'+str(label)) == False):
                            os.makedirs('extract/gtruth_boxes/'+str(label))
                        if (os.path.exists('extract/gtruth_corners/'+str(label)) == False):
                            os.makedirs('extract/gtruth_corners/'+str(label))

                        utils.write_points(points_canonical, pts_name)
                        utils.write_bboxes(box_canonical, box_name)
                        utils.write_bboxes(corners3d[i], corner_name)

            ## Use the bounding boxes to extract the ground truth.

            out = np.asarray(out)
            outlabels = np.asarray(outlabels)

            out_boxes = out[np.where(outlabels==label)]

            if len(out_boxes != 0):
                corners3d = utils.boxes_to_corners_3d(out_boxes)
                points_flag = utils.is_within_3d_box(points, corners3d)

                points_is_within_3d_box = []
                for i in range(len(points_flag)):
                    p = points[points_flag[i]]
                    if len(p)>0:
                        points_is_within_3d_box.append(p)
                        box = out_boxes[i]
                        points_canonical, box_canonical = utils.points_to_canonical(p, box)
                        out_num += 1
                        pts_name = 'extract/out_pts/'+str(label)+'/'+str(out_num)+'.'+element+'.'+file
                        box_name = 'extract/out_boxes/'+str(label)+'/'+str(out_num)+'.'+element+'.'+file
                        corner_name = 'extract/out_corners/'+str(label)+'/'+str(out_num)+'.'+element+'.'+file

                        if (os.path.exists('extract/out_pts/'+str(label)) == False):
                            os.makedirs('extract/out_pts/'+str(label))
                        if (os.path.exists('extract/out_boxes/'+str(label)) == False):
                            os.makedirs('extract/out_boxes/'+str(label))
                        if (os.path.exists('extract/out_corners/'+str(label)) == False):
                            os.makedirs('extract/out_corners/'+str(label))
                            
                        utils.write_points(points_canonical, pts_name)
                        utils.write_bboxes(box_canonical, box_name)
                        utils.write_bboxes(corners3d[i], corner_name)